<a href="https://colab.research.google.com/github/alvarofpinheiro/pifwia_aco/blob/main/PIFWIA_ACO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ACO - Ant Colony Optimization ou Otimização da Colônia de Formigas é uma metaheurística proposta pelo pesquisador Marco Dorigo em 1992 para resolver problemas de otimização inspirado no comportamento das formigas ao saírem de sua colônia para encontrar comida, buscando a solução de problemas computacionais que envolvem procura de caminhos em grafos.

Dorigo, M.; Maniezzo, V.; Colorni, A. Ant system: optimization by a colony of cooperating agents. IEEE Transactions on Systems, Man, and Cybernetics (Volume: 26, Issue: 1). https://doi.org/10.1109/3477.484436, 1996.

In [ ]:
#instala biblioteca Orange Canvas
!pip install Orange3

In [ ]:
#importa bibliotecas
from numpy.random import choice
from scipy import spatial
import matplotlib.pyplot as plt
import random
import math
from sklearn import metrics
import Orange

In [ ]:
#define os hiperparâmetros
DIMENSOES = 2 #determina a quantidade de dimensões do problema
ITERACOES = 200 #quantiddade máxima de ciclos (episódios) especificando quantas explorações podem ser realizadas
COLONIA = 40 #tamanho da população correspondente ao número de formigas no formigueiro
FONTES_ALIMENTACAO = 20 #quantidade de pontos no grafo representando as coordenadas das dimensões do problema
E1 = -5 #extremo esquerdo eixo x
E2 = 5 #extremo direito eixo x
E3 = 0 #extremo inferior eixo y
E4 = 100 #extremo superior eixo y
LIMITES = [E1,E4] #(bound) determina os valores maximos e minimos do espaço de busca (área de busca de fontes de alimentação)
FCUSTO = 'rota' #(fitness) define a aptidão da formiga na possível solução do problema
E = 0.2 #taxa de evaporação do feromônio nos caminhos
ALFA = 0.5 #coeficente de importância da qtde feromônio para determinar probabilidade de escolha do caminho
BETA = 0.7 #coeficente de importância do caminho para determinar a probabilidade de escolha do caminho

In [ ]:
#importa dados
from google.colab import files  
files.upload()

In [ ]:
#instancia objeto de dados com base no caminho gerado na importação do arquivo
dados = Orange.data.Table("/content/dados.csv")

In [ ]:
#explora os metadados e dados da arquivo importado
qtde_campos = len(dados.domain.attributes)
qtde_cont = sum(1 for a in dados.domain.attributes if a.is_continuous)
qtde_disc = sum(1 for a in dados.domain.attributes if a.is_discrete)
print("%d metadados: %d continuos, %d discretos" % (qtde_campos, qtde_cont, qtde_disc))
print("Nome dos metadados:", ", ".join(dados.domain.attributes[i].name for i in range(qtde_campos)),)
dados.domain.attributes #explora os domínios dos atributos (campos da base de dados)
print("Registros:", len(dados)) #explora os dados (quantidade de registros da base de dados)
i = 0 #exibe os primeiros registros para análise dos dados importados
for d in dados[:20]:
  i += 1
  print(i, d)

In [ ]:
#cria arrays das dimensões do problema a ser otimizado
periodo = []
complexidade = [] #1-muito baixa complexidade;2-baixa complexidade;3-média complexidade;4-alta complexidade;e,5-muito alta complexidade
pagina = []
prazo = []
revisao = []
entrega = []
i = 0
for d in dados[:COLONIA]:
  periodo.append(d[1])
  complexidade.append(d[2])
  pagina.append(d[3])
  prazo.append(d[4])
  revisao.append(d[5])
  entrega.append(d[6])
  print("id:",i,"período:",periodo[i],"complexidade:",complexidade[i],"página:",pagina[i],"prazo:",prazo[i],"revisões:",revisao[i],"entrega:",entrega[i])
  per = periodo[i]
  cpx = complexidade[i]
  pag = pagina[i]
  prz = prazo[i]
  rev = revisao[i]
  ent = entrega[i]
  i += 1

In [ ]:
#função custo ou objetivo ou aptidão ou otimização ou fitness - usada para buscar o melhor ponto dentro de um espaço de buscao (melhor global) sem ficar preso em um melhor local
def fitness(problema, posicoes):
  total = 0.0
  if problema == 'rosenbrock':
    for i in range(DIMENSOES-1):
      total += 100*(posicoes[i+1] - posicoes[i]**2)**2 + (1-posicoes[i])**2
  elif problema == 'esfera':
    for i in range(DIMENSOES):
      total += posicoes[i]**2
  elif problema == 'custo':
    for i in range(DIMENSOES-1):
      total += 1 / abs(sum([coord ** 2 for coord in posicao]))
  elif problema == 'rota':
    denominador = sum([(caminho.feromonio)**ALFA * (1 / caminho.comprimento)**BETA for caminho in possiveis_caminhos])
    distribuicao_probabilidades = None
    if denominador == 0:
      distribuicao_probabilidades = [1 / len(possiveis_caminhos)  for _ in possiveis_caminhos]
    else:
      distribuicao_probabilidades = [((caminho.feromonio)**ALFA * (1 / caminho.comprimento)**BETA) / denominador for caminho in possiveis_caminhos]
    total = choice(possiveis_caminhos, 1, p=distribuicao_probabilidades)[0]
  else:
    print('Problema não encontrado!')
  return total

In [ ]:
#formiga - unidade base da otimização, responsável por depositar o feromônio pelos caminhos que percorrem para indicar a qualidade do mesmo
class Formiga:
  def __init__(self, ponto_atual):
    self.ponto_atual = ponto_atual
    self.rota = [ponto_atual]
  def andar(self, ponto):
    self.ponto_atual = ponto
    self.rota.append(ponto)

In [ ]:
#ponto - representação de uma coordenada no espaço
class Ponto:
  def __init__(self, x, y):
    self.x = x
    self.y = y

In [ ]:
#caminho - ligação entre dois pontos
class Caminho:
  def __init__(self, ponto_i, ponto_j):
    self.ponto_i = ponto_i
    self.ponto_j = ponto_j
    self.comprimento = math.sqrt((ponto_i.x - ponto_j.x)**2 + (ponto_i.y - ponto_j.y)**2)
    self.feromonio = 0
    self.formigas_passantes = []
  def contem(self, formiga):
    if self.ponto_i == formiga.ponto_atual:
      return self.ponto_j not in formiga.rota
    elif self.ponto_j == formiga.ponto_atual:
      return self.ponto_i not in formiga.rota
    else:
      return False
  def ponto_adjacente(self, ponto):
    if self.ponto_i == ponto:
      return self.ponto_j
    elif self.ponto_j == ponto:
      return self.ponto_i
    else:
      return None

In [ ]:
#grafo - representação de um conjunto de pontos ligados por caminhos
class Grafo:
  def __init__(self, caminhos):
    self.caminhos = caminhos
    self.melhor_rota = []
    self.comprimento_melhor_rota = 0
  def atualizas_melhor_rota(self, melhor_rota):
    self.melhor_rota = melhor_rota
    self.comprimento_melhor_rota = sum([math.sqrt((i.x - j.x)**2 + (i.y - j.y)**2) for [i, j] in melhor_rota])
  def possiveis_caminhos(self, formiga):
    return [caminho for caminho in self.caminhos if caminho.contem(formiga)]

In [ ]:
#inicializa pontos - fontes de alimentação das formigas que correspondem dimensões do problema
pontos = []
for i in range(FONTES_ALIMENTACAO):
  cpx = complexidade[i]
  pag = pagina[i]
  pontos.append(Ponto(cpx, pag))

In [ ]:
#inicializa caminhos
caminhos = []
i = 0
while i < FONTES_ALIMENTACAO - 1:
  j = i + 1
  while j < FONTES_ALIMENTACAO:
    caminhos.append(Caminho(pontos[i], pontos[j]))
    j += 1
  i += 1

In [ ]:
#posiciona as fontes de alimentação das formigas no espaço de busca
grafo = Grafo(caminhos)
for ponto in pontos:
  plt.plot(ponto.x, ponto.y,  markersize=10, marker='o', color='r')
x = []
y = []
for caminho in caminhos:
  x_i = caminho.ponto_i.x
  x_j = caminho.ponto_j.x
  y_i = caminho.ponto_i.y
  y_j = caminho.ponto_j.y
  x_texto = (x_i + x_j) / 2
  y_texto = (y_i + y_j) / 2
  #plt.text(x_texto, y_texto, "{:.2f}".format(caminho.comprimento))
  x.append(x_i)
  x.append(x_j)
  y.append(y_i)
  y.append(y_j)
print("Plano Cartesiano")
plt.axis([E1,E2,E3,E4])
plt.plot(0,0, marker='*', markersize=10, color='b')
plt.plot(x,y, color='c', linewidth=0.3)
plt.show()
gx = x
gy = y
print("")

In [ ]:
#inicializa população (colonia)
def inicializar_colonia():
  formigas = []
  for _ in range(COLONIA):
    formigas.append(Formiga(random.choice(pontos)))
  return formigas

In [ ]:
#calcula rota (fitness)
def distancia_rota(rota):
  distancia_rota = 0
  for i in range(0, len(rota) - 1):
    distancia = math.sqrt((rota[i].x - rota[i + 1].x)**2 + (rota[i].y - rota[i + 1].y)**2)
    distancia_rota += distancia
  return distancia_rota

In [ ]:
#atualiza o feromônio
def atualizar_feromonios(caminhos):
  for caminho in caminhos:
    soma_heuristica = sum([1 / distancia_rota(formiga.rota) for formiga in caminho.formigas_passantes])
    caminho.feromonio = (1 - E) * caminho.feromonio + soma_heuristica # p = evaporação do feromônico
    caminho.formigas_passantes = []

In [ ]:
#calcula o movimento das formigas
def movimentar_formiga(formiga, grafo):
  while True:
    possiveis_caminhos = grafo.possiveis_caminhos(formiga)
    if possiveis_caminhos == []:
      break
    caminho_escolhido = fitness(possiveis_caminhos)
    caminho_escolhido.formigas_passantes.append(formiga)
    formiga.andar(caminho_escolhido.ponto_adjacente(formiga.ponto_atual))

In [ ]:
#otimiza as rotas
melhor_rota = None
distancia_melhor_sequencia = 0
lista_melhores_valores = []
for _ in range(ITERACOES):
  print("Iteração: {:.0f}".format(_+1))
  formigas = inicializar_colonia()
  for formiga in formigas:
    movimentar_formiga(formiga, grafo)
    if melhor_rota is None or distancia_rota(melhor_rota) > distancia_rota(formiga.rota):
      melhor_rota = formiga.rota
      distancia_melhor_sequencia = distancia_rota(formiga.rota)
      lista_melhores_valores.append(distancia_melhor_sequencia)
  atualizar_feromonios(grafo.caminhos)
  #mostrando a melhor rota a cada iteracao
  plt.axis([E1,E2,E3,E4])
  plt.plot(0,0, marker='*', markersize=10, color='b')
  for ponto in pontos:
    plt.plot(ponto.x, ponto.y, markersize=10, marker='o', color='r')
  x = []
  y = []
  for caminho in caminhos:
    x_i = caminho.ponto_i.x
    x_j = caminho.ponto_j.x
    y_i = caminho.ponto_i.y
    y_j = caminho.ponto_j.y
    x_texto = (x_i + x_j) / 2
    y_texto = (y_i + y_j) / 2
    #plt.text(x_texto, y_texto, "{:.2f}".format(caminho.comprimento))
    x.append(x_i)
    x.append(x_j)
    y.append(y_i)
    y.append(y_j)
  plt.plot(x, y, color='c', linewidth=0.3)
  x = []
  y = []
  for ponto in melhor_rota:
    x.append(ponto.x)
    y.append(ponto.y)
  plt.plot(x, y, color='r', linewidth=0.8)
  plt.show()
  print("Menor Distância: {:.2f} ".format(distancia_melhor_sequencia) + "\n")

In [ ]:
#exibe a curva de convergência das formigas
x = []
y = []
for i in range(len(lista_melhores_valores)):
  x.append(i)
  y.append(lista_melhores_valores[i])
plt.plot(x, y)
plt.title("Curva de Convergência do ACO")
plt.xlabel("Tempo")
plt.ylabel("Iterações")
plt.tight_layout()

In [ ]:
#exibe das posições e otimizações calculadas
posicao_inicial = []
posicao_otimizada = []
for i in range(COLONIA):
  posicao_inicial.append(FORMIGAS[i].posicao)
  posicao_otimizada.append([FORMIGAS[i].fitness,PEIXES[i].posicao[1]])
  print("i:",i,"posição:",posicao_inicial[i],"otimização:",posicao_otimizada[i])
#y = []
#ỹ = []
#for i in range(COLONIA):
#  y.append(int(FORMIGAS[i].posicao[0]))
#  ỹ.append(int(FORMIGAS[i].fitness))
#print('A:', metrics.accuracy_score(y,ỹ))
#print('P:', metrics.precision_score(y,ỹ,average='macro'))
#print('R:', metrics.recall_score(y,ỹ,average='macro'))
#print('F:', metrics.f1_score(y,ỹ,average='macro'))